In [7]:
#instalar libreria optuna
!pip install optuna

El objetivo del programa es encontrar los mejores parámetros para un modelo de Random Forest. Para lograrlo usamos Optuna, que se encarga de explorar y seleccionar las combinaciones que ofrecen el mejor rendimiento.

Optuna usa una optimización estocástica, donde los hiperparámetros se van probando de forma aleatoria o adaptativa según lo que va aprendiendo. Además, emplea optimización bayesiana para modelar cómo se comporta la función objetivo y así dirigir la búsqueda hacia las zonas donde es más probable obtener mejores resultados. A la vez, va descartando los trials que rinden mal, para no seguir perdiendo tiempo en configuraciones poco prometedoras.

In [6]:
# codigo para descubrir los mejores hiperparametros
import pandas as pd
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
import optuna.visualization as vis
import plotly.graph_objects as go

# cargar datos
df = pd.read_csv('dataset.csv')
X = df.drop(['id','target_variable'], axis=1)
y = df['target_variable']

# rangos de busqueda de los hiperparámetros
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 500)
    max_depth = trial.suggest_int('max_depth', 5, 50)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 5, 10)  # mínimo 5 para evitar overfitting
    max_features = trial.suggest_categorical('max_features', ['sqrt','log2', None])

    rf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        class_weight='balanced',
        random_state=42,
        n_jobs=-1  # usamos todos los núcleos
    )

    # 3 folds del cross validation
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    score = cross_val_score(rf, X, y, cv=cv, scoring='accuracy', n_jobs=-1).mean()
    return score

# crear y ejecutar el estudio
study = optuna.create_study(direction='maximize')  # se busca maximizar la acuracy
study.optimize(objective, n_trials=30)  # 30 evaluaciones de hiperparámetros

# filtrar y mostrar solo los resultados con score > 0.8
best_trials = [trial for trial in study.trials if trial.value is not None and trial.value > 0.8]

if best_trials: # comprobar si la lista best_trials no está vacía.

    # ordenar best_trials por su value de mayor a menor.
    best_trials.sort(key=lambda x: x.value, reverse=True)

    print("Trials con accuracy > 0.8:")
    for i, trial in enumerate(best_trials[:3]):  # Mostrar top 3
        print(f"\nTrial #{i+1}:")
        print(f"Score: {trial.value:.4f}")
        print(f"Hiperparámetros: {trial.params}")

    print(f"\nTotal de trials con accuracy > 0.8: {len(best_trials)}")
    print(f"\nMejores hiperparámetros: {study.best_params}")
    print(f"Mejor score CV: {study.best_value:.4f}")

else:
    print("No se encontraron trials con accuracy mayor a 0.8")
    print(f"Mejor score encontrado: {study.best_value:.4f}")


# GRAFICAS

# 1. Parallel Coordinate
fig1 = vis.plot_parallel_coordinate(study)
fig1.update_layout(title="1. Parallel Coordinate: Influencia de Hiperparámetros")
fig1.show()

# 2. Importance Plot
fig2 = vis.plot_param_importances(study)
fig2.update_layout(title="2. Importancia de los Hiperparámetros")
fig2.show()



[I 2025-11-20 18:19:26,841] A new study created in memory with name: no-name-3962c135-8417-4ed3-99a6-7f5ce27db07d
[I 2025-11-20 18:20:07,822] Trial 0 finished with value: 0.7500207767211039 and parameters: {'n_estimators': 467, 'max_depth': 21, 'min_samples_split': 12, 'min_samples_leaf': 8, 'max_features': 'log2'}. Best is trial 0 with value: 0.7500207767211039.
[I 2025-11-20 18:20:30,633] Trial 1 finished with value: 0.7524164347381227 and parameters: {'n_estimators': 303, 'max_depth': 30, 'min_samples_split': 15, 'min_samples_leaf': 8, 'max_features': 'log2'}. Best is trial 1 with value: 0.7524164347381227.
[I 2025-11-20 18:20:47,372] Trial 2 finished with value: 0.7514971564861478 and parameters: {'n_estimators': 316, 'max_depth': 24, 'min_samples_split': 2, 'min_samples_leaf': 8, 'max_features': 'sqrt'}. Best is trial 1 with value: 0.7524164347381227.
[I 2025-11-20 18:21:10,993] Trial 3 finished with value: 0.741496876359475 and parameters: {'n_estimators': 433, 'max_depth': 34, '

Trials con accuracy > 0.8:

Trial #1:
Score: 0.8154
Hiperparámetros: {'n_estimators': 406, 'max_depth': 37, 'min_samples_split': 5, 'min_samples_leaf': 5, 'max_features': None}

Trial #2:
Score: 0.8154
Hiperparámetros: {'n_estimators': 383, 'max_depth': 45, 'min_samples_split': 8, 'min_samples_leaf': 5, 'max_features': None}

Trial #3:
Score: 0.8154
Hiperparámetros: {'n_estimators': 380, 'max_depth': 38, 'min_samples_split': 9, 'min_samples_leaf': 5, 'max_features': None}

Total de trials con accuracy > 0.8: 16

Mejores hiperparámetros: {'n_estimators': 406, 'max_depth': 37, 'min_samples_split': 5, 'min_samples_leaf': 5, 'max_features': None}
Mejor score CV: 0.8154


### Conclusiones

- max_depth es el hiperparámetro más importante.
- max_features es el segundo más importante.
- max_features = None genera casi todos los mejores resultados (>0.81).
- El max_depth óptimo está entre 25 y 35.
- min_samples_leaf = 5 aparece en todos los mejores modelos.
- n_estimators entre 250 y 450 funciona igual que usar 400–500, por lo que tiene poca importancia.
- La accuracy no mejora usando sqrt o log2 como max_features.

---

### Acciones a tomar

- Ajustar los rangos de los hiperparámetros más importantes.
- Reducir max_depth al rango 20–40.
- Fijar max_features = None.
- Fijar min_samples_leaf = 5.
- Reducir el rango de n_estimators, ya que entre 180–240 funciona igual que valores más altos.
- Afinar min_samples_split alrededor de 8–9.


In [8]:
# codigo para descubrir los mejores hiperparametros optimizado por optuna
import pandas as pd
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
import optuna.visualization as vis
import plotly.graph_objects as go

# cargar datos
df = pd.read_csv('dataset.csv')
X = df.drop(['id','target_variable'], axis=1)
y = df['target_variable']

# rangos de busqueda de los hiperparámetros
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 250, 450)         # modificado
    max_depth = trial.suggest_int('max_depth', 20, 50)                 # modificado
    min_samples_split = trial.suggest_int('min_samples_split', 4, 10)  # modificado
    min_samples_leaf = 5                                               # fijado a 5
    max_features = None

    rf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=5,         #fijado
        max_features=None,          #fijado
        class_weight='balanced',
        random_state=42,
        n_jobs=-1  # usamos todos los núcleos
    )

    # 3 folds del cross validation
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    score = cross_val_score(rf, X, y, cv=cv, scoring='accuracy', n_jobs=-1).mean()
    return score

# crear y ejecutar el estudio
study = optuna.create_study(direction='maximize')  # se busca maximizar la acuracy
study.optimize(objective, n_trials=30)  # 30 evaluaciones de hiperparámetros

# filtrar y mostrar solo los resultados con score > 0.8
best_trials = [trial for trial in study.trials if trial.value is not None and trial.value > 0.8]

if best_trials: # comprobar si la lista best_trials no está vacía.

    # ordenar best_trials por su value de mayor a menor.
    best_trials.sort(key=lambda x: x.value, reverse=True)

    print("Trials con accuracy > 0.8:")
    for i, trial in enumerate(best_trials[:3]):  # Mostrar top 3
        print(f"\nTrial #{i+1}:")
        print(f"Score: {trial.value:.4f}")
        print(f"Hiperparámetros: {trial.params}")

    print(f"\nTotal de trials con accuracy > 0.8: {len(best_trials)}")
    print(f"\nMejores hiperparámetros: {study.best_params}")
    print(f"Mejor score CV: {study.best_value:.4f}")

else:
    print("No se encontraron trials con accuracy mayor a 0.8")
    print(f"Mejor score encontrado: {study.best_value:.4f}")


# GRAFICAS

# 1. Parallel Coordinate
fig1 = vis.plot_parallel_coordinate(study)
fig1.update_layout(title="1. Parallel Coordinate: Influencia de Hiperparámetros")
fig1.show()

# 2. Importance Plot
fig2 = vis.plot_param_importances(study)
fig2.update_layout(title="2. Importancia de los Hiperparámetros")
fig2.show()



[I 2025-11-20 18:41:01,794] A new study created in memory with name: no-name-cff2587e-1470-42f9-9158-58cdc2ad3cd9
[I 2025-11-20 18:42:34,137] Trial 0 finished with value: 0.8116380974937937 and parameters: {'n_estimators': 426, 'max_depth': 20, 'min_samples_split': 6}. Best is trial 0 with value: 0.8116380974937937.
[I 2025-11-20 18:43:54,370] Trial 1 finished with value: 0.8151757756695698 and parameters: {'n_estimators': 396, 'max_depth': 43, 'min_samples_split': 4}. Best is trial 1 with value: 0.8151757756695698.
[I 2025-11-20 18:44:58,398] Trial 2 finished with value: 0.814562939796141 and parameters: {'n_estimators': 325, 'max_depth': 40, 'min_samples_split': 4}. Best is trial 1 with value: 0.8151757756695698.
[I 2025-11-20 18:45:48,426] Trial 3 finished with value: 0.8142286663182147 and parameters: {'n_estimators': 251, 'max_depth': 46, 'min_samples_split': 5}. Best is trial 1 with value: 0.8151757756695698.
[I 2025-11-20 18:46:53,016] Trial 4 finished with value: 0.814730056748

Trials con accuracy > 0.8:

Trial #1:
Score: 0.8158
Hiperparámetros: {'n_estimators': 366, 'max_depth': 35, 'min_samples_split': 8}

Trial #2:
Score: 0.8157
Hiperparámetros: {'n_estimators': 364, 'max_depth': 31, 'min_samples_split': 9}

Trial #3:
Score: 0.8157
Hiperparámetros: {'n_estimators': 360, 'max_depth': 38, 'min_samples_split': 8}

Total de trials con accuracy > 0.8: 30

Mejores hiperparámetros: {'n_estimators': 366, 'max_depth': 35, 'min_samples_split': 8}
Mejor score CV: 0.8158


Aunque en cada iteración Optuna va proponiendo combinaciones de hiperparámetros que tienden a mejorar el rendimiento del modelo, en este proyecto todavía voy ajustando manualmente los rangos y comprobando los resultados visualizando las gráficas generadas.

Soy consciente de que todo este proceso se puede automatizar por completo (ajuste dinámico de rangos, relanzar estudios según resultados previos, selección automática de hiperparámetros, etc.). Lo implementaré en futuros proyectos, pero en esta versión he preferido mantener control manual para entender mejor cómo se comporta el modelo.

Si queréis ver las gráficas, podéis descargar el programa y ejecutarlo en Colab, donde sí se renderizan porque el código corre y activa los módulos interactivos. En GitHub no aparecen porque el visor es estático. También estan subidas en el repositorio.